In [49]:
# https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification

# Install Additional Dependencies

In [2]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

# Disable WANDB

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, \
     DataCollatorWithPadding, pipeline
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import  Dataset
import pandas as pd
import matplotlib.pyplot as plt
import evaluate

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Utilize BERT Model from HuggingFace
from transformers import BertTokenizer, BertModel

# Load uncased
model_BERT_base = BertModel.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# load the bert-base uncased tokenizer
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# tokenize a sequence
tokenizer_sentence=tokenizer_bert.encode('AI has been my friend')
tokenizer_sentence

[101, 9932, 2038, 2042, 2026, 2767, 102]

In [8]:
import torch


# runing tokens via the model
response = model_BERT_base(torch.tensor(tokenizer_sentence).unsqueeze(0))

# Embedding for each token
response.last_hidden_state

tensor([[[ 0.0065,  0.0303, -0.1594,  ..., -0.1599,  0.1518,  0.3864],
         [-0.2074, -0.4378,  0.0418,  ..., -0.2403, -0.0033,  0.4402],
         [ 0.2448, -0.3865, -0.2682,  ..., -0.0998,  0.0463,  0.6762],
         ...,
         [ 0.2216,  0.2247,  0.6810,  ...,  0.0474, -0.0571,  0.0918],
         [-0.3868, -0.4962,  0.1083,  ...,  0.7687,  0.1917,  0.4949],
         [ 0.6903,  0.0883, -0.1104,  ...,  0.1298, -0.7293, -0.4013]]],
       grad_fn=<NativeLayerNormBackward0>)

# Fine Tuning BERT

In [9]:
from transformers import BertForMaskedLM, pipeline

nlp = pipeline("fill-mask", model='bert-base-cased')

print(type(nlp.model))

preds = nlp(f"If you don’t know how to swim, you will  {nlp.tokenizer.mask_token} in this lake.")

print('If you don’t know how to swim, you will .... in this lake.')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
If you don’t know how to swim, you will .... in this lake.
Token:drown. Score: 72.56%
Token:die. Score: 23.95%
Token:be. Score: 0.63%
Token:drowned. Score: 0.45%
Token:fall. Score: 0.39%


# Roberta-Base

In [10]:
nlp = pipeline("fill-mask", model='roberta-base') # Using a flavor of BERT called Roberta

preds = nlp(f"If you don’t know how to swim, you will  {nlp.tokenizer.mask_token} in this lake.")

print('If you don’t know how to swim, you will .... in this lake.')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


If you don’t know how to swim, you will .... in this lake.
Token: drown. Score: 89.65%
Token: die. Score: 8.90%
Token: perish. Score: 0.38%
Token: survive. Score: 0.19%
Token: starve. Score: 0.18%


# Distil Roberta

In [11]:
nlp = pipeline("fill-mask", model='distilroberta-base') # Using a flavor of BERT called Distilroberta

print(type(nlp.model))

preds = nlp(f"If you don’t know how to swim, you will  {nlp.tokenizer.mask_token} in this lake.")

print('If you don’t know how to swim, you will .... in this lake.')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
If you don’t know how to swim, you will .... in this lake.
Token: drown. Score: 99.15%
Token: swim. Score: 0.34%
Token: die. Score: 0.24%
Token: perish. Score: 0.03%
Token: be. Score: 0.02%


# Distil Bert

In [12]:
nlp = pipeline("fill-mask", model='distilbert-base-cased')  # Using a flavor of BERT called DistilBERT

preds = nlp(f"If you don’t know how to swim, you will  {nlp.tokenizer.mask_token} in this lake.")

print('If you don’t know how to swim, you will .... in this lake.')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


If you don’t know how to swim, you will .... in this lake.
Token:drown. Score: 78.49%
Token:swim. Score: 12.65%
Token:die. Score: 2.16%
Token:stay. Score: 0.66%
Token:float. Score: 0.65%


In [13]:
import pandas as pd

data_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bert Sentiment Analysis/WELFake_Dataset.csv')

In [14]:
# reduce dataset becuase its too big



# Total number of rows in the dataset
N = len(data_news)

# Total sample size is 10% of the dataset
sample_size = int(0.1 * N)

# To have an equal number of samples for labels 0 and 1:
samples_per_label = sample_size // 2

# Optional: set a random state for reproducibility
random_state = 42

# Sample from each label group separately
data_news_label0 = data_news[data_news['label'] == 0].sample(n=samples_per_label, random_state=random_state)
data_news_label1 = data_news[data_news['label'] == 1].sample(n=samples_per_label, random_state=random_state)

# Concatenate the two samples to form the balanced reduced dataset
df_reduced = pd.concat([data_news_label0, data_news_label1]).sample(frac=1, random_state=random_state).reset_index(drop=True)

# Display the resulting DataFrame and label counts to verify
print(df_reduced.head())
print(df_reduced['label'].value_counts())

   Unnamed: 0                                              title  \
0       66730  No country for migrant stowaways caught on fer...   
1       26251  Russia says bellicose rhetoric on North Korea ...   
2       41871  AWESOME! SEAN SPICER Gives Trump’s Salary Away...   
3       56352  Reaching Peak Greek at the Met Museum - The Ne...   
4       18481  Trump Weighs Cuts to Coast Guard, T.S.A. and F...   

                                                text  label  
0  COPENHAGEN (Reuters) - Twelve migrants, appare...      0  
1  MOSCOW (Reuters) - Russian Foreign Minister Se...      0  
2  Zinke and Brandyburg accompanied Spicer to the...      1  
3  Seen from on high, history is a grand sorting ...      0  
4  WASHINGTON  —   The Trump administration is co...      0  
label
0    3606
1    3606
Name: count, dtype: int64


In [15]:
data_news = df_reduced

In [16]:
# data_news['str_label'] = data_news['label'].apply(lambda x: 'FAKE' if x == 1 else 'REAL')

In [17]:
data_news = data_news.dropna()

# Segment Parses

In [18]:
# This code segment parses the data_news dataset into a more manageable format

titles = []
tokenized_titles = []
sequence_labels = data_news['label']

title, tokenized_title =  [], []
for news in data_news['title']:
    title.append(news)
    tokenized_title.append(news.split(' '))



In [19]:
# Python list for each news
title[0], tokenized_title[0], sequence_labels[0]

('No country for migrant stowaways caught on ferry between Ukraine and Turkey',
 ['No',
  'country',
  'for',
  'migrant',
  'stowaways',
  'caught',
  'on',
  'ferry',
  'between',
  'Ukraine',
  'and',
  'Turkey'],
 'REAL')

In [20]:
#Use distilbert model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

In [21]:
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['REAL', 'FAKE']

In [22]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

print(f'There are {len(unique_sequence_labels)} unique sequence labels')

There are 2 unique sequence labels


In [23]:
print(tokenized_title[0])
print(title[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['No', 'country', 'for', 'migrant', 'stowaways', 'caught', 'on', 'ferry', 'between', 'Ukraine', 'and', 'Turkey']
No country for migrant stowaways caught on ferry between Ukraine and Turkey
0
REAL


# Split Data

In [24]:
news_dataset = Dataset.from_dict(
    dict(
        titles=title,
        label=sequence_labels,
        tokens=tokenized_title,
    )
)
news_dataset = news_dataset.train_test_split(test_size=0.2)

news_dataset

DatasetDict({
    train: Dataset({
        features: ['titles', 'label', 'tokens'],
        num_rows: 5732
    })
    test: Dataset({
        features: ['titles', 'label', 'tokens'],
        num_rows: 1433
    })
})

In [25]:
news_dataset['train'][0]

{'titles': 'Local Man Continues Love Affair With Sound Of Own Voice',
 'label': 1,
 'tokens': ['Local',
  'Man',
  'Continues',
  'Love',
  'Affair',
  'With',
  'Sound',
  'Of',
  'Own',
  'Voice']}

# Tokenizer

In [26]:
from transformers import AutoTokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [27]:
def preprocess_function(examples):
    return tokenizer(examples["titles"], truncation=True) # truncation=True makes sure to exludes instances with more
                                                            # 512 tokens|

In [28]:
# go over all our data set, tokenize them
seq_clf_tokenized_news = news_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5732 [00:00<?, ? examples/s]

Map:   0%|          | 0/1433 [00:00<?, ? examples/s]

In [29]:
news_dataset

DatasetDict({
    train: Dataset({
        features: ['titles', 'label', 'tokens'],
        num_rows: 5732
    })
    test: Dataset({
        features: ['titles', 'label', 'tokens'],
        num_rows: 1433
    })
})

# Batch of Data


In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                                         num_labels=len(unique_sequence_labels),)

# set an index -> label dictionary
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
sequence_clf_model.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "REAL",
    "1": "FAKE"
  },
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.47.1",
  "vocab_size": 30522
}

In [33]:
sequence_clf_model.config.id2label[0]

'REAL'

In [34]:
# metric = load_metric("accuracy")
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):  # common method to take in logits and calculate accuracy of the eval set
    logits, labels = eval_pred   # logit and label are returning from training loop
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels) # compute the accuracy

import evaluate
metric = evaluate.load("accuracy")
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#####################################################

def compute_metrics_binary(eval_pred):
    """metrics for binary classification"""

    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # Calculate the AUC score
    auc_score = roc_auc_score(labels, preds)

    # Calculate the accuracy, true positive, false positive, false negative, and true negative values
    acc = metric.compute(predictions=preds, references=labels)
    tp = ((preds >= 0.5) & (labels == 1)).sum()
    fp = ((preds >= 0.5) & (labels == 0)).sum()
    fn = ((preds < 0.5) & (labels == 1)).sum()
    tn = ((preds < 0.5) & (labels == 0)).sum()

    # Calculate the precision, recall, and F1 score
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return {
        'Validation Accuracy': acc['accuracy'],
        'Validation Precision': auc_score,
        'Validation AUC': precision,
        'Validation Recall': recall,
        'Validation F1_Score': f1_score,
        'Validation TP': tp,
        'Validation FP': fp,
        'Validation FN': fn,
        'Validation TN': tn,
    }

#####################################################

from sklearn.metrics import classification_report

def compute_metrics_multiclass(eval_pred):
    """metrics for multiclass classification"""
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    report = classification_report(labels, preds, output_dict=True)
    acc_score = report['accuracy']
    pre_score = report['macro avg']['precision']
    rcl_score = report['macro avg']['recall']
    f1_score = report['macro avg']['f1-score']

    return {
        'Validation Accuracy': acc_score,
        'Validation Macro Recall': rcl_score,
        'Validation Macro Precision': pre_score,
        'Validation Macro F1_Score': f1_score,
        }

In [35]:
epochs = 2

# Training argument
training_args = TrainingArguments(
    output_dir="./news_clf/results", # Local directory to save check point of our model as fitting
    num_train_epochs=epochs,         # minimum of two epochs
    per_device_train_batch_size=32,  # batch size for training and evaluation, it common to take around 32,
    per_device_eval_batch_size=32,   # sometimes less or more, The smaller batch size, the more change model update
    load_best_model_at_end=True,     # Even if we overfit the model by accident, load the best model through checkpoint

    # some deep learning parameters that the trainer is able to take in
    warmup_steps = len(seq_clf_tokenized_news['train']) // 5,  # learning rate scheduler by number of warmup steps
    weight_decay = 0.05,    # weight decay for our learning rate schedule (regularization)

    logging_steps = 1,  # Tell the model minimum number of steps to log between (1 means logging as much as possible)
    log_level = 'info',
    evaluation_strategy = 'epoch', # It is "steps" or "epoch", we choose epoch: how many times to stop training to test
    eval_steps = 50,
    save_strategy = 'epoch'  # save a check point of our model after each epoch
)

# Define the trainer:
trainer = Trainer(
    model=sequence_clf_model,   # take our model (sequence_clf_model)
    args=training_args,         # we just set it above
    train_dataset=seq_clf_tokenized_news['train'], # training part of dataset
    eval_dataset=seq_clf_tokenized_news['test'],   # test (evaluation) part of dataset
    compute_metrics=compute_metrics_binary,    # This part is optional but we want to calculate accuracy of our model
    data_collator=data_collator         # data colladior with padding. Infact, we may or may not need a data collator
                                        # we can check the model to see how it lookes like with or without the collator
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [36]:
# Get initial metrics: evaluation on test set
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32


{'eval_loss': 0.6984493732452393,
 'eval_model_preparation_time': 0.0027,
 'eval_Validation Accuracy': 0.471039776692254,
 'eval_Validation Precision': 0.4503645869967879,
 'eval_Validation AUC': 0.4992,
 'eval_Validation Recall': 0.8253968253968254,
 'eval_Validation F1_Score': 0.6221335992023929,
 'eval_Validation TP': 624,
 'eval_Validation FP': 626,
 'eval_Validation FN': 132,
 'eval_Validation TN': 51,
 'eval_runtime': 100.7919,
 'eval_samples_per_second': 14.217,
 'eval_steps_per_second': 0.446}

In [37]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,732
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 360
  Number of trainable parameters = 66,955,010


Epoch,Training Loss,Validation Loss,Model Preparation Time,Validation accuracy,Validation precision,Validation auc,Validation recall,Validation f1 Score,Validation tp,Validation fp,Validation fn,Validation tn
1,0.865700,0.320593,0.002700,0.868807,0.869101,0.884824,0.863757,0.874163,653,85,103,592
2,0.193400,0.227095,0.002700,0.905094,0.904959,0.912234,0.907407,0.909814,686,66,70,611


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32


Saving model checkpoint to ./news_clf/results/checkpoint-180
Configuration saved in ./news_clf/results/checkpoint-180/config.json
Model weights saved in ./news_clf/results/checkpoint-180/model.safetensors
Saving model checkpoint to ./news_clf/results/checkpoint-360
Configuration saved in ./news_clf/results/checkpoint-360/config.json
Model weights saved in ./news_clf/results/checkpoint-360/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32
Saving model checkpoint to ./news_clf/results/checkpoint-360
Configuration saved in ./news_clf/results/checkpoint-360/config.json
Model weights saved in ./news_clf/results/checkpoint-360/model.safetensors


Training completed. D

TrainOutput(global_step=360, training_loss=0.4231121548348003, metrics={'train_runtime': 2679.2172, 'train_samples_per_second': 4.279, 'train_steps_per_second': 0.134, 'total_flos': 111234323513616.0, 'train_loss': 0.4231121548348003, 'epoch': 2.0})

In [38]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32


{'eval_loss': 0.22709499299526215,
 'eval_model_preparation_time': 0.0027,
 'eval_Validation Accuracy': 0.9050942079553385,
 'eval_Validation Precision': 0.9049592428469828,
 'eval_Validation AUC': 0.9122340425531915,
 'eval_Validation Recall': 0.9074074074074074,
 'eval_Validation F1_Score': 0.9098143236074272,
 'eval_Validation TP': 686,
 'eval_Validation FP': 66,
 'eval_Validation FN': 70,
 'eval_Validation TN': 611,
 'eval_runtime': 91.5193,
 'eval_samples_per_second': 15.658,
 'eval_steps_per_second': 0.492,
 'epoch': 2.0}

In [39]:
# make a pipline by passing in our fine-tuned model with tokenizer
pipe = pipeline("text-classification", model=sequence_clf_model, tokenizer=tokenizer)
pipe('Please add Here We Go by Dispatch to my road trip playlist')

Device set to use cpu


[{'label': 'FAKE', 'score': 0.899931788444519}]

In [40]:
# We can save our model on drirectory we specified
trainer.save_model()

Saving model checkpoint to ./news_clf/results
Configuration saved in ./news_clf/results/config.json
Model weights saved in ./news_clf/results/model.safetensors


In [41]:
pipe = pipeline("text-classification", "./news_clf/results", tokenizer=tokenizer)

loading configuration file ./news_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./news_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "REAL",
    "1": "FAKE"
  },
  "initializer_range": 0.02,
  "label2id": null,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.47.1",
  "vocab_size": 30522
}

loading configuration file ./news_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./news_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "a

In [42]:
text = 'The Battle of New York: Why This Primary Matters'
pipe(text)

[{'label': 'FAKE', 'score': 0.630978524684906}]

In [43]:
text = """Breaking News: Researchers have discovered a new species of dinosaur that
        can breathe fire. The creature, named Pyrodino, is believed to have lived
        during the Jurassic period and could shoot flames out of its nostrils,
        making it one of the deadliest predators of its time."""
pipe(text)

[{'label': 'FAKE', 'score': 0.9417099356651306}]

In [44]:
for param in sequence_clf_model.distilbert.parameters():
    param.requires_grad = False   # "False" makes the parameters unable to update. "grad" stands for gradient
                                  # it never upgrade during training

In [45]:
epochs = 2

# Training argument
training_args = TrainingArguments(
    output_dir="./news_clf/results", # Local directory to save check point of our model as fitting
    num_train_epochs=epochs,         # minimum of two epochs
    per_device_train_batch_size=32,  # batch size for training and evaluation, it common to take around 32,
    per_device_eval_batch_size=32,   # sometimes less or more, The smaller batch size, the more change model update
    load_best_model_at_end=True,     # Even if we overfit the model by accident, load the best model through checkpoint

    # some deep learning parameters that the trainer is able to take in
    warmup_steps = len(seq_clf_tokenized_news['train']) // 5,  # learning rate scheduler by number of warmup steps
    weight_decay = 0.05,    # weight decay for our learning rate schedule (regularization)

    logging_steps = 1,  # Tell the model minimum number of steps to log between (1 means logging as much as possible)
    log_level = 'info',
    evaluation_strategy = 'epoch', # It is "steps" or "epoch", we choose epoch: how many times to stop training to test
    eval_steps = 50,
    save_strategy = 'epoch'  # save a check point of our model after each epoch
)

# Define the trainer:
trainer = Trainer(
    model=sequence_clf_model,   # take our model (sequence_clf_model)
    args=training_args,         # we just set it above
    train_dataset=seq_clf_tokenized_news['train'], # training part of dataset
    eval_dataset=seq_clf_tokenized_news['test'],   # test (evaluation) part of dataset
    compute_metrics=compute_metrics_binary,    # This part is optional but we want to calculate accuracy of our model
    data_collator=data_collator         # data colladior with padding. Infact, we may or may not need a data collator
                                        # we can check the model to see how it lookes like with or without the collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [46]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32


{'eval_loss': 0.22709499299526215,
 'eval_model_preparation_time': 0.0031,
 'eval_Validation Accuracy': 0.9050942079553385,
 'eval_Validation Precision': 0.9049592428469828,
 'eval_Validation AUC': 0.9122340425531915,
 'eval_Validation Recall': 0.9074074074074074,
 'eval_Validation F1_Score': 0.9098143236074272,
 'eval_Validation TP': 686,
 'eval_Validation FP': 66,
 'eval_Validation FN': 70,
 'eval_Validation TN': 611,
 'eval_runtime': 115.1006,
 'eval_samples_per_second': 12.45,
 'eval_steps_per_second': 0.391}

In [47]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,732
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 360
  Number of trainable parameters = 592,130


Epoch,Training Loss,Validation Loss,Model Preparation Time,Validation accuracy,Validation precision,Validation auc,Validation recall,Validation f1 Score,Validation tp,Validation fp,Validation fn,Validation tn
1,0.092600,0.230851,0.003100,0.903001,0.903284,0.916329,0.898148,0.907148,679,62,77,615
2,0.086400,0.241884,0.003100,0.902303,0.902468,0.913978,0.899471,0.906667,680,64,76,613


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32


Saving model checkpoint to ./news_clf/results/checkpoint-180
Configuration saved in ./news_clf/results/checkpoint-180/config.json
Model weights saved in ./news_clf/results/checkpoint-180/model.safetensors
Saving model checkpoint to ./news_clf/results/checkpoint-360
Configuration saved in ./news_clf/results/checkpoint-360/config.json
Model weights saved in ./news_clf/results/checkpoint-360/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: titles, tokens. If titles, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1433
  Batch size = 32
Saving model checkpoint to ./news_clf/results/checkpoint-360
Configuration saved in ./news_clf/results/checkpoint-360/config.json
Model weights saved in ./news_clf/results/checkpoint-360/model.safetensors


Training completed. D

TrainOutput(global_step=360, training_loss=0.14533298451246487, metrics={'train_runtime': 964.8484, 'train_samples_per_second': 11.882, 'train_steps_per_second': 0.373, 'total_flos': 111234323513616.0, 'train_loss': 0.14533298451246487, 'epoch': 2.0})

In [57]:
pipe = pipeline("text-classification", model=sequence_clf_model, tokenizer=tokenizer)
text = """
China, Canada and Mexico vow swift response to Trump tariffs
"""
pipe(text)

Device set to use cpu


[{'label': 'REAL', 'score': 0.9849724173545837}]